# Aerial Scene Classification using SIFT + LBP + SVM

## 1. Import Libraries and Configuration

In [ ]:
# Import required libraries
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report, confusion_matrix
import seaborn as sns

# Configuration
data_dir = './Aerial_Landscapes'
img_size = (256, 256)
pca_dimention = 64
svm_C = 10
svm_kernel = 'rbf'
test_size = 0.2
random_seed = 666

## 2. Load Class Names

In [ ]:
# Load class names from dataset folder
class_names = sorted(os.listdir(data_dir))
if '.DS_Store' in class_names:
    class_names.remove('.DS_Store')
print("running program...")

all_descriptors = []
image_descriptor_list = [] 
y_labels = []

sift = cv2.SIFT_create()

for label, class_name in enumerate(class_names):
    class_path = os.path.join(data_dir, class_name)
    for file in tqdm(os.listdir(class_path), desc=f"handling with class: {class_name}"):
        img_path = os.path.join(class_path, file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, img_size)

        # get the SIFT descriptor
        keypoints, descriptors = sift.detectAndCompute(img, None)
        if descriptors is not None:
            all_descriptors.append(descriptors)
            image_descriptor_list.append(descriptors)
            y_labels.append(label)


## 3. Apply PCA for Dimensionality Reduction

In [ ]:
# Apply PCA to reduce dimensionality of SIFT descriptors

all_descriptors_stacked = np.vstack(all_descriptors)

pca = PCA(n_components=pca_dimention)
pca.fit(all_descriptors_stacked)

X_pca_mean = []

for descriptors in image_descriptor_list:
    descriptors_pca = pca.transform(descriptors)  
    desc_mean = np.mean(descriptors_pca, axis=0)  
    X_pca_mean.append(desc_mean)


X = np.array(X_pca_mean)
y = np.array(y_labels)


## 4. Normalize Features and Split Dataset

In [ ]:
# Normalize data and split into train/test sets
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=666, stratify=y)


## 5. Train SVM Classifier

In [ ]:
# Train an SVM classifier using RBF kernel
# training
print("\n training SVM model")
clf = SVC(kernel='rbf', C=10, gamma='scale')  
clf.fit(X_train, y_train)



## 6. Evaluate Model Performance

In [ ]:

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=class_names))
print("Model Accuracy: ", accuracy_score(y_test, y_pred))

print("\nModel report:")
print(classification_report(y_test, y_pred, target_names=class_names))

precision = precision_score(y_test, y_pred, average='macro') 
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1-score  : {f1:.4f}")



## 7. Plot Confusion Matrix

In [ ]:

model_confusion_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(model_confusion_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('Reality Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(
    clf, X_pca_mean, y, cv=5, scoring='accuracy', train_sizes=np.linspace(0.01, 1.0, 100)
)

train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

plt.plot(train_sizes, train_mean, 'o-', label='training data accuracy')
plt.plot(train_sizes, test_mean, 'o-', label='testing data accuracy')
plt.xlabel('Amount of training data')
plt.ylabel('Accuracy')
plt.title('Training Curve')
plt.legend()
plt.grid()
plt.show()
